# IMD Radar Data

The India Meteorological Department (IMD) provides radar data in a format similar to IRIS/Sigmet, but with some differences in structure and formatting. `radarx` offers specialized support for processing and analyzing this IMD radar data, ensuring compatibility and ease of use despite the variations from standard IRIS/Sigmet formats.

In [ ]:
import os
import subprocess
import radarx as rx

## Load IMD Radar Data

In [ ]:
# Base URL for the radar data files on GitHub
base_url = (
    "https://raw.githubusercontent.com/syedhamidali/pyscancf_examples/main/data/goa16/"
)

# List of radar data files to download
files = [
    "GOA210516024101-IMD-B.nc",
    "GOA210516024101-IMD-B.nc.1",
    "GOA210516024101-IMD-B.nc.2",
    "GOA210516024101-IMD-B.nc.3",
    "GOA210516024101-IMD-B.nc.4",
    "GOA210516024101-IMD-B.nc.5",
    "GOA210516024101-IMD-B.nc.6",
    "GOA210516024101-IMD-B.nc.7",
    "GOA210516024101-IMD-B.nc.8",
    "GOA210516024101-IMD-B.nc.9",
]

# Target directory to save downloaded files
target_dir = os.path.join(os.getenv("GITHUB_WORKSPACE", "."), "radarx_data")
os.makedirs(target_dir, exist_ok=True)


# Function to download files using curl
def download_with_curl(file_name):
    file_url = base_url + file_name
    file_path = os.path.join(target_dir, file_name)

    if not os.path.exists(file_path):
        print(f"Downloading {file_name}...")
        subprocess.run(["curl", "-o", file_path, file_url], check=True)
        print(f"Downloaded {file_name}")
    else:
        print(f"{file_name} already exists.")

    return file_path


# Download the files
downloaded_files = []
for file_name in files:
    file_path = download_with_curl(file_name)
    downloaded_files.append(file_path)

In [ ]:
print(f"Number of files: {len(downloaded_files)}")

## Read a Sweep

In [ ]:
# Load the first radar file using radarx
swp = rx.io.read_sweep(downloaded_files[0])
swp

In [ ]:
# This test ensures that the fixed angle matches the expected value (required for pytest)
import numpy as np

np.testing.assert_almost_equal(swp.fixed_angle.values, 0.1977539)

We have successfully read one sweep in the above cell. Now, let's create a volume scan.

## Create a Volume

We have a total of 10 files, each representing a different sweep. Our goal is to combine these individual sweeps into a single CF-Radial volume scan dataset.

In [ ]:
for file in downloaded_files:
    print(os.path.basename(file))

In [ ]:
vol = rx.io.read_volume(downloaded_files)

In [ ]:
vol

We have successfully generated a volume, which can now be explored further. The volume is stored in a DataTree, a high-level structure that organizes and manages the data within the xarray framework.

In [ ]:
vol.groups

In [ ]:
ds = vol["volume_0"].ds

In [ ]:
ds

We have successfully created a volume, and we can export it to CF-Radial formatted NetCDF using `xarray`.

In [ ]:
# We can save all these cfradial1 files by iterating over the vol object

outdir = "IMD_OUT"
os.makedirs(outdir, exist_ok=True)
for key in vol.children:
    ds = vol[key].ds
    time_str = ds.time.min().dt.strftime("%Y%m%d_%H%M%S").item()
    ds.to_netcdf(os.path.join(outdir, f"GOA_{time_str}_cfrad1.nc"))

In [ ]:
os.listdir("IMD_OUT/")

## Convert to CF/Radial2

We can also convert it to cfradial2 data using to_cfradial2 function

In [ ]:
dtree = rx.io.to_cfradial2_volumes(vol)

In [ ]:
dtree.groups

In [ ]:
ds = dtree["volume_0"].ds

In [ ]:
# Export all the data in a dtree

# for volume in dtree:
#     dtree[volume].to_netcdf(filename.nc)
# or
#     dtree[volume].to_zarr(filename.zarr)

In [ ]:
import xarray as xr

# Create a notebook cell with a simple test:
assert 1 + 1 == 2

# Expected tuple
expected_groups = ("/", "/volume_0")

# Use xr.testing.assert_equal to check if vol.groups matches the expected tuple
xr.testing.assert_equal(
    xr.DataArray(list(vol.groups)), xr.DataArray(list(expected_groups))
)